In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from dataset2 import TranslationDataset, Vocab
from submission import get_bleu
import torch

from config import filenames

In [8]:
import zipfile
import os

zip_path = '../bhw2-data.zip'
extract_to = '../'
data_folder = extract_to + 'data/'
train_src_filename = data_folder + 'train.de-en.de'
train_trg_filename = data_folder + 'train.de-en.en'
test_src_filename = data_folder + 'val.de-en.de'
test_trg_filename = data_folder + 'val.de-en.en'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print(f"Files extracted to {extract_to}")
print("Extracted files:", os.listdir(data_folder))

Files extracted to ../
Extracted files: ['test1.de-en.de', 'train.de-en.de', 'train.de-en.en', 'val.de-en.de', 'val.de-en.en']


In [9]:
device = 'cpu'

In [10]:
from dataset2 import Vocab

In [11]:
vocab_src = Vocab(filenames['train_src'], min_freq=5)
vocab_trg = Vocab(filenames['train_trg'], min_freq=5)

In [12]:
print(len(vocab_src))
print(len(vocab_trg))

24461
18144


In [13]:
vocab_src.max_freq_sub

8

In [14]:
word = 'avcs'
len(word)

4

In [16]:
src = 'ich bin nba schpiler'.split()
trg = 'i want to play in nba'.split()

In [25]:
vocab_src.word_counter['ich']

56752

In [19]:
vocab_src.check_sub('nba')

True

In [26]:
src_seq = vocab_src.encode(src)
trg_seq = vocab_trg.encode(trg)

In [28]:
vocab_trg.decode(trg_seq, src=src, vocab_src=vocab_src)

[6]
['nba']


['i', 'want', 'to', 'play', 'in', 'nba']

In [ ]:
train_dataset = TranslationDataset(vocab_src, 
                                   vocab_trg, 
                                   train_src_filename, 
                                   train_trg_filename, 
                                   max_len=64, 
                                   device=device)
test_dataset = TranslationDataset(vocab_src, 
                                  vocab_trg, 
                                  test_src_filename, 
                                  test_trg_filename, 
                                  max_len=72, 
                                  device=device, 
                                  sort_lengths=True)

 25%|██▍       | 48392/195915 [00:08<00:25, 5843.12it/s]


KeyboardInterrupt: 

In [ ]:
print(*train_dataset[110], sep='\n')

tensor([    2, 10700, 14316, 15972, 18322, 20704,  3977,  7637,    15,  7876,
        36391, 22284,    39,     3], device='cuda:0')
tensor([    2, 23615,  1849, 15907,  1685, 13823, 23615,    35,  8570, 23582,
        13823, 23580, 25556,  3471,    39,     3], device='cuda:0')


In [ ]:
for i, (src, trg) in enumerate(train_dataset):
    if i % 10_000 == 0:
        print(len(src), len(trg))

15 15
29 36
24 25
23 24
18 24
21 20
9 8
17 15
18 18
20 21
11 10
23 25
10 9
7 10
29 25
12 12
10 10
17 22
10 9
34 41


In [ ]:
from dataset import TrainDataLoader, TestDataLoader
unk_idx, pad_idx, bos_idx, eos_idx = 0, 1, 2, 3

train_loader = TrainDataLoader(train_dataset, shuffle=True)
test_loader = TestDataLoader(test_dataset)

In [ ]:
import gc
def clear_cache(model):
    model.cpu()
    del model
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
for i, batch in enumerate(train_loader):
   if i% 500 == 0:
      print(batch[0].size(), batch[1].size())

torch.Size([64, 48]) torch.Size([64, 48])
torch.Size([64, 48]) torch.Size([64, 48])
torch.Size([64, 48]) torch.Size([64, 48])
torch.Size([64, 48]) torch.Size([64, 48])
torch.Size([64, 48]) torch.Size([64, 48])
torch.Size([64, 44]) torch.Size([64, 43])
torch.Size([64, 48]) torch.Size([64, 48])


In [ ]:
# clear_cache(model)

In [ ]:
import torch.nn as nn
from lstm2 import LSTM_2, train, plot_losses
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

src_vocab_size = len(vocab_src)
trg_vocab_size = len(vocab_trg)
embedding_dim = 64
hidden_size = 128
learning_rate = 5e-3
gamma = 0.1
num_epochs = 12
weight_decay = 1e-5
label_smoothing = 0.1

model = LSTM_2(
    src_vocab_size=src_vocab_size,
    trg_vocab_size=trg_vocab_size,
    embedding_dim=embedding_dim,
    hidden_size=hidden_size,
    dropout=0.15
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx, label_smoothing=label_smoothing)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = ReduceLROnPlateau(optimizer, patience=2, factor=gamma, verbose=True)
print(model)

LSTM_2(
  (src_embedding): Embedding(38465, 64)
  (trg_embedding): Embedding(26257, 64)
  (encoder): LSTM(64, 128, num_layers=2, batch_first=True, dropout=0.15, bidirectional=True)
  (encoder_output_proj): Linear(in_features=256, out_features=128, bias=True)
  (decoder): LSTM(64, 128, num_layers=2, batch_first=True, dropout=0.15)
  (encoder_hidden_proj): ModuleList(
    (0-1): 2 x Linear(in_features=256, out_features=128, bias=True)
  )
  (encoder_cell_proj): ModuleList(
    (0-1): 2 x Linear(in_features=256, out_features=128, bias=True)
  )
  (fc): Linear(in_features=256, out_features=26257, bias=True)
)


c:\Users\Dubinin Daniil\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
# train_losses, val_losses = train(model, optimizer, num_epochs, train_loader, test_loader, criterion, trg_vocab_size, scheduler)
# print(*torch.load('lstm-regularized-12epoch.pt').keys(), sep='\n')
model.load('lstm-regularized-cut-vocab-10epoch.pt')
train_losses, val_losses = model.train_loss, model.val_loss
plot_losses(train_losses, val_losses)

RuntimeError: Error(s) in loading state_dict for LSTM_2:
	size mismatch for src_embedding.weight: copying a param with shape torch.Size([25379, 64]) from checkpoint, the shape in current model is torch.Size([38465, 64]).
	size mismatch for trg_embedding.weight: copying a param with shape torch.Size([19057, 64]) from checkpoint, the shape in current model is torch.Size([26257, 64]).
	size mismatch for fc.weight: copying a param with shape torch.Size([19057, 256]) from checkpoint, the shape in current model is torch.Size([26257, 256]).
	size mismatch for fc.bias: copying a param with shape torch.Size([19057]) from checkpoint, the shape in current model is torch.Size([26257]).

In [ ]:
print(train_losses, val_losses)

[3.90864683 2.87150702 2.56812275 2.4015559  2.29022142 2.20895368
 2.14491234 2.09421665 2.05214971 2.01697366 1.98539129 1.95969942] [2.92919451 2.57247037 2.47399157 2.40769878 2.38161317 2.37686232
 2.35697159 2.36191532 2.35576686 2.36135063 2.36614612 2.3800391 ]


In [ ]:
results0 = {
    'model' : 'LSTM(num_layers=2, dropout=0.1, embdeing_dim=64)',
    'oprimizer': 'Adam(lr=0.001)',
    'train loss': 1.6788,
    'val loss': 2.3845,
    'bleu4 local, public': (24.61, 20.36),
    'parameteres': 15459071,
    'epochs': 11,
    'weight_file': 'weights/lstm-11epochs.pt',
    'time/epoch, total time': '~3m, ~40m',
    'vocab sizes': (55315, 34047)
}

In [ ]:
results1 = {
    'model' : 'LSTM(num_layers=2, dropout=0.15, embdeing_dim=64)',
    'oprimizer': 'AdamW(lr=5e-3, weight_decay=1e-5)',
    'train loss': 1.95969942,
    'val loss': 2.3800391,
    'bleu4 local, public': (15.87, ),
    'parameteres': ,
    'epochs': 12,
    'weight_file': 'weights/lstm-11epochs.pt',
    'time/epoch, total time': '3m 10s, 40m',
    'vocab sizes': (38465, 34047)
}

SyntaxError: invalid syntax (1309727638.py, line 6)

In [ ]:
from dataset import SubmissionDataset, SubmissionDataLoader

val_filename = data_folder + 'test1.de-en.de'

submission_dataset = SubmissionDataset(val_filename, vocab_src)
submission_loader = SubmissionDataLoader(submission_dataset)

In [ ]:
get_bleu(model, test_loader, vocab_trg)

100%|██████████| 4/4 [00:10<00:00,  2.71s/it]


17.36

In [ ]:
from submission import make_submission
make_submission(model, submission_loader, train_dataset)

100%|██████████| 24/24 [00:18<00:00,  1.33it/s]


In [ ]:
model.parameters

<bound method Module.parameters of LSTM_Deep(
  (src_embedding): Embedding(55315, 64)
  (trg_embedding): Embedding(34047, 64)
  (encoder): LSTM(64, 128, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (encoder_output_proj): Linear(in_features=256, out_features=128, bias=True)
  (decoder): LSTM(64, 128, num_layers=2, batch_first=True, dropout=0.1)
  (encoder_hidden_proj): ModuleList(
    (0-1): 2 x Linear(in_features=256, out_features=128, bias=True)
  )
  (encoder_cell_proj): ModuleList(
    (0-1): 2 x Linear(in_features=256, out_features=128, bias=True)
  )
  (fc): Linear(in_features=256, out_features=34047, bias=True)
)>